### Extract Grantees' 990 Form

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re
import xml.etree.ElementTree as ET
import xmltodict
import urllib.request
import json
from pandas.io.json import json_normalize

In [2]:
# import 990 index file to connect EIN with S3 url
files = glob.glob('Z:/Projects/Grants/Data_Updates/990 Form/990_index_grantees/*.xlsx')
grantee990_index = {}
for f in files:
    grantee990_index[os.path.splitext(os.path.basename(f))[0].split('_')[-1]] = pd.read_excel(f).drop(['Unnamed: 0'], axis=1)


In [5]:
grantee990_index.keys()

dict_keys(['index13', 'index14', 'index15', 'index16', 'index17', 'index18'])

In [25]:
key_list = list(grantee990_index.keys())
for i in range(len(key_list)):
    print(key_list[i], grantee990_index[key_list[i]].shape)

index13 (16353, 8)
index14 (16836, 8)
index15 (16958, 8)
index16 (15836, 8)
index17 (16006, 8)
index18 (8160, 8)


In [6]:
# check 
grantee990_index['index18'].shape

(8160, 8)

In [3]:
# pre-defined check function, return bool
def check_re(pat, S):
    return bool(re.search(pat, S))

In [4]:
# pre-define: col_info: pattern to use when parseing: nested list
# re-new: basic info added

colinfo = [["Filer", "EIN"],
           "TaxPeriodEndDt",
           "TaxYr",
           ["Filer", "BusinessNameLine1Txt"],
           ["Filer", "BusinessNameLine2Txt"],
           ["Filer", "BusinessNameControlTxt"],
           ["Filer", "CityNm"],
           ["Filer", "StateAbbreviationCd"],
           "ActivityOrMissionDesc",
           ["NetAssetsOrFundBalances", "EOYAmt"],
           "TotalAssetsEOYAmt",
           "CYTotalRevenueAmt", 
           "TotalLiabilitiesEOYAmt", 
           "CYTotalExpensesAmt",
           "CYContributionsGrantsAmt",
           ["CashNonInterestBearingGrp", "EOYAmt"],
           ["SavingsAndTempCashInvstGrp", "EOYAmt"],
           ["PledgesAndGrantsReceiveableGrp", "EOYAmt"],
           ["AccountsReceivableGrp", "EOYAmt"],
           ["ReceivablesFromOfficersEtcGrp", "EOYAmt"],
           ["RcvblFromDisqualifiedPrsnGrp", "EOYAmt"],
           ["OthNotesLoansReceivableNetGrp", "EOYAmt"],
           ["InventoriesForSaleOrUseGrp", "EOYAmt"],
           ["PrepaidExpensesDefrdChargesGrp", "EOYAmt"],
           "AccountsPayableAccrExpnssGrp",
           ["GrantsPayableGrp", "EOYAmt"],
           ["DeferredRevenueGrp", "EOYAmt"],
           ["EscrowAccountLiabilityGrp", "EOYAmt"],
           ["LoansFromOfficersDirectorsGrp", "EOYAmt"],
           ["OtherLiabilitiesGrp", "EOYAmt"],
           ["DepreciationDepletionGrp", "TotalAmt"],
           ["TotalFunctionalExpensesGrp", "ProgramServicesAmt"],
           "TotalGrossReceiptsAmt"
          ]

In [10]:
# function version 2: pass index-df instead of year
def retrieve_grantee990_dfv2(index_df):
    """
    input: index_df
    
    requirements: 990-grantees intersection index df
                  pre-defined check_re function return bool
                  pre-defined list of col info to use as pattern (regx)
    notes: 
    
    """
    #index_key = "index" + year[-2:]
    #index_df = grantee990_index[index_key]
    from xml.parsers.expat import ExpatError
    from urllib.error import URLError, HTTPError
    
    grantee_990df = pd.DataFrame()
    
    for x in index_df["URL"]:
        try:
            tree = ET.ElementTree(file=urllib.request.urlopen(x))
        except URLError:
            pass
        
        root = tree.getroot()
        xmlstr = ET.tostring(root, encoding='utf8', method='xml')
        
        try:
            row = json_normalize(dict(xmltodict.parse(xmlstr)))
        except ExpatError:
            pass
        
        # select columns: ANY
        ## re.search pattern
        bool_df = pd.DataFrame()
        for i in colinfo:
            if type(i) == str:
                pat = "(?=.*{})".format(i)              
            else:
                pat = ''
                for j in i:
                    a = "(?=.*{})".format(j)
                    pat = pat + a
            
            # return bool for re.search
            bool_row = [check_re(pat, col) for col in list(row.columns)]
            bool_df = pd.concat([bool_df, pd.Series(bool_row)], axis=1)
            
        boolfilter = bool_df.any(axis=1).tolist() # row-wise
        
        row_filtered = row.iloc[:, bool_df.loc[boolfilter, :].index]
        grantee_990df = pd.concat([grantee_990df, row_filtered], axis=0)
    
    return grantee_990df
    

In [8]:
index_df18_3000 = grantee990_index["index18"].iloc[:3000, ]
grantee990_18_3000 = retrieve_grantee990_dfv2(index_df18_3000)
print(grantee990_18_3000.shape)

(3000, 38)


In [15]:
pd.options.display.max_columns = 40
grantee990_18_3000[:5]

,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:AccountsPayableAccrExpnssGrp.ns0:BOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:AccountsPayableAccrExpnssGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:AccountsReceivableGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:ActivityOrMissionDesc,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:CYContributionsGrantsAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:CYTotalExpensesAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:CYTotalRevenueAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:CashNonInterestBearingGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:DeferredRevenueGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:DepreciationDepletionGrp.ns0:TotalAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:EscrowAccountLiabilityGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:GrantsPayableGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:InventoriesForSaleOrUseGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:LoansFromOfficersDirectorsGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:NetAssetsOrFundBalancesEOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:OthNotesLoansReceivableNetGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:OtherLiabilitiesGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:PrepaidExpensesDefrdChargesGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:RcvblFromDisqualifiedPrsnGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:ReceivablesFromOfficersEtcGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:SavingsAndTempCashInvstGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:TotalAssetsEOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:TotalFunctionalExpensesGrp.ns0:ProgramServicesAmt,ns0:Return.ns0:ReturnData.ns0:IRS990.ns0:TotalLiabilitiesEOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990EZ.ns0:NetAssetsOrFundBalancesEOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990EZ.ns0:NetAssetsOrFundBalancesGrp.ns0:EOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990PF.ns0:Form990PFBalanceSheetsGrp.ns0:TotalAssetsEOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990PF.ns0:Form990PFBalanceSheetsGrp.ns0:TotalLiabilitiesEOYAmt,ns0:Return.ns0:ReturnData.ns0:IRS990ScheduleG.ns0:TotalGrossReceiptsAmt,ns0:Return.ns0:ReturnHeader.ns0:Filer.ns0:BusinessName.ns0:BusinessNameLine1Txt,ns0:Return.ns0:ReturnHeader.ns0:Filer.ns0:BusinessName.ns0:BusinessNameLine2Txt,ns0:Return.ns0:ReturnHeader.ns0:Filer.ns0:BusinessNameControlTxt,ns0:Return.ns0:ReturnHeader.ns0:Filer.ns0:EIN,ns0:Return.ns0:ReturnHeader.ns0:Filer.ns0:ForeignAddress.ns0:CityNm,ns0:Return.ns0:ReturnHeader.ns0:Filer.ns0:USAddress.ns0:CityNm,ns0:Return.ns0:ReturnHeader.ns0:Filer.ns0:USAddress.ns0:StateAbbreviationCd,ns0:Return.ns0:ReturnHeader.ns0:TaxPeriodEndDt,ns0:Return.ns0:ReturnHeader.ns0:TaxYr
0,12140407,12814147,5363939,TO CREATE PUBLIC SCHOOLS THAT EQUIP EDUCATIONA...,62815001,73827353,69907277,NaN,95225,845278,NaN,NaN,NaN,NaN,50178621,2024157,7377370,457160,NaN,NaN,30189412,70465363,61066951,20286742,NaN,NaN,NaN,NaN,NaN,KIPP FOUNDATION,NaN,KIPP,943362724,NaN,SAN FRANCISCO,CA,2018-06-30,2017
0,1213110,864269,1556674,TO PROVIDE SOCIAL SERVICES FOR THE NEEDS OF IN...,17133526,17889434,18884881,1864325,NaN,69628,NaN,NaN,0,NaN,1027228,0,1853902,0,0,0,0,3895399,16579254,2868171,NaN,NaN,NaN,NaN,NaN,CATHOLIC FAMILY AND COMMUNITY,SERVICES INC,CATH,221487121,NaN,PATERSON,NJ,2018-06-30,2017
0,686587,575400,NaN,TO PROVIDE HOLISTIC AND CLIENT CENTERED SERVIC...,4618043,11193181,11409331,1046674,NaN,382417,NaN,NaN,2000,NaN,6715445,NaN,NaN,89807,NaN,NaN,439640,9279644,9234898,2564199,NaN,NaN,NaN,NaN,NaN,ODYSSEY HOUSE LOUISIANA INC,NaN,ODYS,720743677,NaN,NEW ORLEANS,LA,2018-06-30,2017
0,3363767,3597523,5452474,TO STRENGTHEN AND PRESERVE HEALTHY KINSHIP FAM...,36803527,36853394,36803893,1495,325000,294095,NaN,NaN,5703,NaN,4081056,NaN,NaN,262547,NaN,NaN,3269,8641307,30607734,4560251,NaN,NaN,NaN,NaN,NaN,A SECOND CHANCE INC,NaN,ASEC,251729710,NaN,PITTSBURGH,PA,2018-06-30,2017
0,205520,548456,NaN,DEVELOPING THE CAPACITY OF STU

In [65]:
bool_df = pd.DataFrame()
for i in colinfo:
    if type(i) == str:
        pat = "(?=.*{})".format(i)              
    else:
        pat = ''
        for j in i:
            a = "(?=.*{})".format(j)
            pat = pat + a
    print(pat)
# return bool for re.search
    bool_row = [check_re(pat, col) for col in list(grantee990_18_100.columns)]
    bool_df = pd.concat([bool_df, pd.Series(bool_row)], axis=1)
boolfilter = bool_df.any(axis=1).tolist() # row-wise
             


(?=.*NetAssetsOrFundBalances)(?=.*EOYAmt)
(?=.*TotalAssets)(?=.*EOYAmt)
(?=.*CYTotalRevenueAmt)
(?=.*TotalLiabilities)(?=.*EOYAmt)
(?=.*CYTotalExpensesAmt)
(?=.*CYContributionsGrantsAmt)
(?=.*CashNonInterestBearingGrp)(?=.*EOYAmt)
(?=.*SavingsAndTempCashInvstGrp)(?=.*EOYAmt)
(?=.*PledgesAndGrantsReceiveableGrp)(?=.*EOYAmt)
(?=.*AccountsReceivableGrp)(?=.*EOYAmt)
(?=.*ReceivablesFromOfficersEtcGrp)(?=.*EOYAmt)
(?=.*RcvblFromDisqualifiedPrsnGrp)(?=.*EOYAmt)
(?=.*OthNotesLoansReceivableNetGrp)(?=.*EOYAmt)
(?=.*InventoriesForSaleOrUseGrp)(?=.*EOYAmt)
(?=.*PrepaidExpensesDefrdChargesGrp)(?=.*EOYAmt)
(?=.*AccountsPayableAccrExpnssGrp)
(?=.*GrantsPayableGrp)(?=.*EOYAmt)
(?=.*DeferredRevenueGrp)(?=.*EOYAmt)
(?=.*EscrowAccountLiabilityGrp)(?=.*EOYAmt)
(?=.*LoansFromOfficersDirectorsGrp)(?=.*EOYAmt)
(?=.*OtherLiabilitiesGrp)(?=.*EOYAmt)
(?=.*DepreciationDepletionGrp)(?=.*TotalAmt)
(?=.*TotalFunctionalExpensesGrp)(?=.*ProgramServicesAmt)


In [ ]:
pat = '(?=.*AccountsReceivableGrp)(?=.*EOYAmt)'
pat = "(?=.*{})(?=.*{})".format(colinfo[1][0], colinfo[1][1])

for i in colinfo:
    if type(i) == str:
        pat = "(?=.*{})".format(i)
        print(pat)
              
    else:
        a = ''
        for j in i:
            pat = "(?=.*{})".format(j)
            a = a + pat
            print(a)

In [ ]:
df_try = pd.DataFrame()
for x in sample['URL']:
    tree = ET.ElementTree(file=urllib.request.urlopen(x))
    root = tree.getroot()
    xmlstr = ET.tostring(root, encoding='utf8', method='xml')
    df_append = json_normalize(dict(xmltodict.parse(xmlstr)))
    
    df_try = pd.concat([df_try, df_append], axis=0)